<a href="https://colab.research.google.com/github/aa05420/Coal-Mine-Warning-and-Monitoring-System/blob/main/ACGAN_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import os
# define the path to your data folders
data_path = '/content/drive/MyDrive/MineData'
images_haz_path = os.path.join(data_path, "images_haz")
images_nonhaz_path = os.path.join(data_path, "images_nonhaz")

# define the image size
IMG_SIZE = 64

# use ImageDataGenerator to load and preprocess the data
datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.2)

train_generator = datagen.flow_from_directory(directory=data_path,
                                              target_size=(IMG_SIZE, IMG_SIZE),
                                              class_mode='binary',
                                              batch_size=32,
                                              subset='training')

# normalize the images to the range [-1, 1]
X_train = (train_generator.next()[0].astype(np.float32) - 0.5) * 2.0

Found 798 images belonging to 2 classes.


In [1]:
import os
import numpy as np
from PIL import Image
import keras
from keras.layers import Input, Dense, Reshape, Flatten, BatchNormalization, Activation, Embedding, multiply
from keras.layers import UpSampling2D, Conv2D, LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam


import matplotlib.pyplot as plt
import math
import sys
import numpy as np

data_path = '/content/drive/MyDrive/MineData'
images_haz_path = os.path.join(data_path, "images_haz")
images_nonhaz_path = os.path.join(data_path, "images_nonhaz")

# Load and preprocess haz images
haz_images = []
for filename in os.listdir(images_haz_path):
    img_path = os.path.join(images_haz_path, filename)
    with Image.open(img_path) as img:
        img = img.convert('L')  # convert to grayscale
        img = img.resize((28, 28))
        img = np.array(img)
        img = (img.astype(np.float32) - 127.5) / 127.5  # normalize to [-1, 1]
        haz_images.append(img)

# Load and preprocess nonhaz images
nonhaz_images = []
for filename in os.listdir(images_nonhaz_path):
    img_path = os.path.join(images_nonhaz_path, filename)
    with Image.open(img_path) as img:
        img = img.convert('L')  # convert to grayscale
        img = img.resize((28, 28))
        img = np.array(img)
        img = (img.astype(np.float32) - 127.5) / 127.5  # normalize to [-1, 1]
        nonhaz_images.append(img)


# Combine haz and nonhaz images
X_train = np.concatenate([np.array(haz_images), np.array(nonhaz_images)])

# Create labels for haz and nonhaz images
y_haz = np.zeros(len(haz_images))
y_nonhaz = np.ones(len(nonhaz_images))
y_train = np.concatenate([y_haz, y_nonhaz])

# Shuffle the data
shuffle_idxs = np.random.permutation(len(X_train))
X_train = X_train[shuffle_idxs]
y_train = y_train[shuffle_idxs]

NUM_CLASSES = 2
NUM_EPOCHS = 20
BATCH_SIZE = 32
BATCH_COUNT = math.ceil(X_train.shape[0] / BATCH_SIZE)
NOISE_DIM = 100
HALF_BATCH = int(BATCH_SIZE / 2)

adam = Adam(lr=2e-4, beta_1=0.5)
losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

noise = Input(shape=(NOISE_DIM,))
label = Input(shape=(1,), dtype='int32')
label_embedding = Flatten()(Embedding(NUM_CLASSES, NOISE_DIM)(label))
generator_input = multiply([noise, label_embedding])

x = Dense(7 * 7 * 128)(generator_input)
x = Reshape((7, 7, 128))(x)
x = LeakyReLU(0.2)(x)
x = BatchNormalization()(x)
x = UpSampling2D()(x)
x = Conv2D(64, kernel_size=(5, 5), padding='same')(x)
x = LeakyReLU(0.2)(x)
x = BatchNormalization()(x)
x = UpSampling2D()(x)
generator_output = Conv2D(1, kernel_size=(5, 5), padding='same', activation='tanh')(x)

generator = Model([noise, label], generator_output)

generator.compile(loss='binary_crossentropy', optimizer=adam)

img = Input(shape=(28, 28, 1))
x = Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', input_shape=(28, 28, 3))(img)
x = LeakyReLU(0.2)(x)
x = Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same')(x)
x = LeakyReLU(0.2)(x)
x = Flatten()(x)
validity = Dense(1, activation='sigmoid')(x)
class_label = Dense(NUM_CLASSES + 1, activation='sigmoid')(x)
# another class for fake class

discriminator = Model(img, [validity, class_label])

discriminator.compile(loss=losses, optimizer=adam)

discriminator.trainable = False
noise = Input(shape=(NOISE_DIM,))
label = Input(shape=(1,), dtype='int32')
generated_img = generator([noise, label])
valid, target_label = discriminator(generated_img)
combined = Model([noise, label], [valid, target_label])
combined.compile(loss=losses, optimizer=adam)

def save_imgs(epoch, num_examples=100):
    noise = np.random.normal(0, 1, size=[num_examples, NOISE_DIM])
    sampled_labels = np.random.randint(0, 2, num_examples).reshape(-1, 1)

    generated_imgs = generator.predict([noise, sampled_labels])
    generated_imgs = generated_imgs.reshape(num_examples, 28, 28)
    for i in range(num_examples):
        
    # plt.figure(figsize=(10,10))
        filename = '/content/drive/MyDrive/images/generated_image_epoch_{0}_index_{1}.png'.format(epoch+1, i+1)
        img = Image.fromarray((generated_imgs[i] * 255).astype(np.uint8))
        img.save(filename)
    #     plt.subplot( 10, 10, i+1)
    #     plt.imshow(generated_imgs[i], interpolation='nearest', cmap='gray')
    #     plt.axis('off')
    # plt.tight_layout()
    # plt.savefig('/content/drive/MyDrive/images/acgan_generated_image_epoch_{0}.png'.format(epoch + 1))

for epoch in range(NUM_EPOCHS):
    epoch_d_loss = 0.
    epoch_g_loss = 0.

    for step in range(BATCH_COUNT):
        # ---------------------
        #  Train Discriminator
        # ---------------------

        # randomly select half a batch of real images
        idx = np.random.randint(0, X_train.shape[0], HALF_BATCH)
        imgs = X_train[idx]

        # generate half a batch of fake images
        noise = np.random.normal(0, 1, (HALF_BATCH, NOISE_DIM))
        sampled_labels = np.random.randint(0, 2, HALF_BATCH).reshape(-1)
        generated_imgs = generator.predict([noise, sampled_labels])

        # use one-sided label smoothing
        real_valid_y = np.ones((HALF_BATCH, 1)) * 0.9
        fake_valid_y = np.zeros((HALF_BATCH, 1))

        real_class_labels = y_train[idx]
        # real classes are [0, NUM_CLASSES-1]
        fake_class_labels = NUM_CLASSES * np.ones(HALF_BATCH).reshape(-1)

        # train on real and fake images
        d_loss_real = discriminator.train_on_batch(imgs, [real_valid_y, real_class_labels])
        d_loss_fake = discriminator.train_on_batch(generated_imgs, [fake_valid_y, fake_class_labels])
        # use combined loss
        d_loss = 0.5 * (np.array(d_loss_real) + np.array(d_loss_fake))

        epoch_d_loss += np.mean(d_loss)

        # ---------------------
        #  Train Generator
        # ---------------------
        noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_DIM))
        real_valid_y = np.ones((BATCH_SIZE, 1))
        sampled_labels = np.random.randint(0, 2, BATCH_SIZE).reshape(-1)
        g_loss = combined.train_on_batch([noise, sampled_labels], [real_valid_y, sampled_labels])

        # fold loss into cumulative moving average
        # # use combined loss
        epoch_g_loss += np.mean(g_loss)

    print("%d [D loss: %f] [G loss: %f]" % ((epoch + 1), epoch_d_loss / BATCH_COUNT, epoch_g_loss / BATCH_COUNT))

    if (epoch + 1) % 10 == 0:
        generator.save('/content/drive/MyDrive/models/acgan_generator_{0}.h5'.format(epoch + 1))
        save_imgs(epoch)


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


ValueError: ignored